“GenMapSapper.ipynb” - создать карту для игры “сапер”
Создать функцию(или класс), которая вернет карту для игры Сапер в зависимости от входящих параметров.


In [ ]:
// SapperMapGenerator.cs
using System;
using System.Collections.Generic;
using System.Linq;

public static class SapperMapGenerator
{
    public static string[,]? GenerateSapperMap(
        int height = 9,
        int width = 9,
        int desiredRandomMines = 10, // колво мин
        IEnumerable<(int r, int c)>? specificMinesParam = null, // коллекция кортежей с координатами мин
        (int r, int c)? firstClickCoords = null) // Координаты (y,x) первого хода, эта клетка должна быть точно без мины
    {
        var mineLocations = new HashSet<(int r, int c)>();
        int actualMinesToPlace;

        // 1. Определение местоположений мин и их фактического количества
        if (specificMinesParam != null && specificMinesParam.Any())
        {
            mineLocations.UnionWith(specificMinesParam); // Добавляем указанные мины
            actualMinesToPlace = mineLocations.Count;

            // Проверка: клетка первого хода не должна быть среди явно указанных мин
            if (firstClickCoords.HasValue && mineLocations.Contains(firstClickCoords.Value))
            {
                Console.WriteLine($"Ошибка: Клетка первого хода ({firstClickCoords.Value.r},{firstClickCoords.Value.c}) " +
                                  "указана как мина в 'specificMinesParam' и не может быть безопасной.");
                return null;
            }
        }
        else
        {
            // Случайная расстановка мин
            actualMinesToPlace = desiredRandomMines;
            if (actualMinesToPlace < 0) actualMinesToPlace = 0; // Минимальное количество мин - 0

            // Предварительные проверки на невозможность генерации
            if (actualMinesToPlace > height * width)
            {
                Console.WriteLine("Ошибка: Запрошенное количество случайных мин превышает общее количество клеток.");
                return null;
            }
            if (firstClickCoords.HasValue && actualMinesToPlace >= height * width)
            {
                // Если все клетки должны быть минами, безопасный первый ход невозможен
                Console.WriteLine("Ошибка: Невозможно гарантировать безопасный первый ход, если все клетки должны быть минами.");
                return null;
            }

            var availableCells = new List<(int r, int c)>();
            for (int r = 0; r < height; r++)
            {
                for (int c = 0; c < width; c++)
                {
                    // Исключаем клетку первого хода из кандидатов на минирование
                    if (firstClickCoords.HasValue && firstClickCoords.Value.r == r && firstClickCoords.Value.c == c)
                    {
                        continue;
                    }
                    availableCells.Add((r, c));
                }
            }

            if (availableCells.Count < actualMinesToPlace)
            {
                Console.WriteLine("Ошибка: Недостаточно доступных клеток для расстановки запрошенного количества случайных мин (учитывая первый ход).");
                return null;
            }

            Random random = new Random();
            for (int i = 0; i < actualMinesToPlace; i++)
            {
                int randomIndex = random.Next(availableCells.Count);
                mineLocations.Add(availableCells[randomIndex]);
                availableCells.RemoveAt(randomIndex); // Удаляем, чтобы избежать дублирования
            }
        }

        // 2. Инициализация и заполнение доски минами
        string[,] board = new string[height, width];
        for (int r = 0; r < height; r++)
        {
            for (int c = 0; c < width; c++)
            {
                if (mineLocations.Contains((r, c)))
                {
                    board[r, c] = "*";
                }
                else
                {
                    board[r, c] = "0"; // Временное значение, будет обновлено количеством мин
                }
            }
        }

        // 3. Подсчет мин вокруг каждой не-минной клетки
        for (int r = 0; r < height; r++)
        {
            for (int c = 0; c < width; c++)
            {
                if (board[r, c] == "*") // Пропускаем сами мины
                {
                    continue;
                }

                int mineCount = 0;
                // Обход 8 соседних клеток
                for (int dr = -1; dr <= 1; dr++) // Смещение по строке
                {
                    for (int dc = -1; dc <= 1; dc++) // Смещение по столбцу
                    {
                        if (dr == 0 && dc == 0) continue; // Это сама текущая клетка

                        int nr = r + dr; // Координаты соседа
                        int nc = c + dc;

                        // Проверка на выход за пределы поля
                        if (nr >= 0 && nr < height && nc >= 0 && nc < width)
                        {
                            if (board[nr, nc] == "*") // Если сосед - мина
                            {
                                mineCount++;
                            }
                        }
                    }
                }
                board[r, c] = mineCount.ToString();
            }
        }
        return board;
    }

    // Вспомогательная функция для вывода карты сапёра в консоль.
    public static void PrintSapperMap(string[,]? board)
    {
        if (board == null)
        {
            Console.WriteLine("--- Карта не была сгенерирована ---");
            return;
        }
        int height = board.GetLength(0);
        int width = board.GetLength(1);

        for (int r = 0; r < height; r++)
        {
            for (int c = 0; c < width; c++)
            {
                // Форматируем вывод для выравнивания: "{значение,2}" означает выделить 2 символа, выровнять по правому краю.
                // Добавляем пробел после каждого элемента для лучшего разделения.
                Console.Write(string.Format("{0,2}", board[r, c]) + " ");
            }
            Console.WriteLine();
        }

        if (width > 0 && height > 0)
        {
            // Каждый элемент "XX ", т.е. 3 символа. Общая ширина линии (width * 3) минус последний пробел.
            Console.WriteLine(new string('-', width * 3 - 1));
        }
    }
}

In [8]:
Console.OutputEncoding = Encoding.UTF8;

Console.WriteLine("Карта по умолчанию (9x9, 10 мин):");
var defaultMap = SapperMapGenerator.GenerateSapperMap(); // Это оператор верхнего уровня
SapperMapGenerator.PrintSapperMap(defaultMap);

Console.WriteLine("\nМаленькая карта (5x5, 5 мин):");
var smallMap = SapperMapGenerator.GenerateSapperMap(height: 5, width: 5, desiredRandomMines: 5);
SapperMapGenerator.PrintSapperMap(smallMap);

Console.WriteLine($"\nКарта 5x5 с точно заданными минами:");
var specificMinesCoords = new List<(int r, int c)> { (0, 2), (1, 1), (2, 4), (3, 0), (4, 3) };
var specificMap = SapperMapGenerator.GenerateSapperMap(height: 5, width: 5, specificMinesParam: specificMinesCoords);
SapperMapGenerator.PrintSapperMap(specificMap);

Console.WriteLine("\nКарта 3x3, 8 мин (все кроме одной), первый ход в (1,1) безопасен:");
var almostAllMinesMap = SapperMapGenerator.GenerateSapperMap(height: 3, width: 3, desiredRandomMines: 8, firstClickCoords: (1, 1));
SapperMapGenerator.PrintSapperMap(almostAllMinesMap);
if (almostAllMinesMap != null)
{
    Console.WriteLine($"Значение в (1,1): {almostAllMinesMap[1, 1]} (должно быть 0, если вокруг нет мин)");
    }

Карта по умолчанию (9x9, 10 мин):
 0  1  *  1  1  *  2  1  1 
 1  2  1  1  1  1  2  *  1 
 *  1  0  0  0  0  2  2  2 
 1  1  0  0  0  1  2  *  2 
 0  0  1  1  1  1  *  4  * 
 0  0  1  *  1  1  2  *  2 
 0  0  1  1  1  0  2  2  2 
 0  0  0  0  0  0  1  *  1 
 0  0  0  0  0  0  1  1  1 
--------------------------

Маленькая карта (5x5, 5 мин):
 0  1  3  *  2 
 0  1  *  *  3 
 0  1  2  3  * 
 1  1  1  1  1 
 1  *  1  0  0 
--------------

Карта 5x5 с точно заданными минами:
 1  2  *  1  0 
 1  *  2  2  1 
 2  2  1  1  * 
 *  1  1  2  2 
 1  1  1  *  1 
--------------

Карта 3x3, 8 мин (все кроме одной), первый ход в (1,1) безопасен:
 *  *  * 
 *  8  * 
 *  *  * 
--------
Значение в (1,1): 8 (должно быть 0, если вокруг нет мин)
